# Login to Confluent Cloud

In [ ]:
%%bash
export CCLOUD_EMAIL=
export CCLOUD_PASSWORD=
export PATH=$PATH:ccloud
ccloud login
ccloud environment list

In [2]:
%%bash
export PATH=$PATH:ccloud
ccloud environment use 'env-9gyvm'
ccloud kafka cluster list

Now using "env-9gyvm" as the default (active) environment.
      Id      |     Name      | Type  | Provider |     Region      | Availability | Status  
+-------------+---------------+-------+----------+-----------------+--------------+--------+
    lkc-w0xqj | participant22 | BASIC | gcp      | asia-southeast2 | single-zone  | UP      
    lkc-mnyd2 | participant21 | BASIC | gcp      | asia-southeast2 | single-zone  | UP      
  * lkc-0k719 | admin         | BASIC | gcp      | asia-southeast2 | single-zone  | UP      


# Create ksqlDB app

In [5]:
%%bash
export PATH=$PATH:ccloud
# ccloud ksql app create adminksqlapp --cluster lkc-0k719

+--------------+--------------------------------------------------------------+
| Id           | lksqlc-rgo80                                                 |
| Name         | adminksqlapp                                                 |
| Topic Prefix | pksqlc-w7q3m                                                 |
| Kafka        | lkc-0k719                                                    |
| Storage      |                                                          500 |
| Endpoint     | https://pksqlc-w7q3m.asia-southeast2.gcp.confluent.cloud:443 |
| Status       | PROVISIONING                                                 |
+--------------+--------------------------------------------------------------+


(DEPRECATED) In a future release, api-key and api-secret will be required flags when creating a ksql cluster.


In [1]:
%%bash
export PATH=$PATH:ccloud
ccloud ksql app list

       Id      |     Name     | Topic Prefix |   Kafka   | Storage |                           Endpoint                           | Status  
+--------------+--------------+--------------+-----------+---------+--------------------------------------------------------------+--------+
  lksqlc-x08ox | ksqlDB_app_2 | pksqlc-8193r | lkc-0k719 |     500 | https://pksqlc-8193r.asia-southeast2.gcp.confluent.cloud:443 | UP      
  lksqlc-v92pn | ksqlDB_app_1 | pksqlc-0x1v2 | lkc-0k719 |     500 | https://pksqlc-0x1v2.asia-southeast2.gcp.confluent.cloud:443 | UP      


In [4]:
%%bash
export PATH=$PATH:ccloud
ccloud ksql app describe lksqlc-v92pn

+--------------+--------------------------------------------------------------+
| Id           | lksqlc-v92pn                                                 |
| Name         | ksqlDB_app_1                                                 |
| Topic Prefix | pksqlc-0x1v2                                                 |
| Kafka        | lkc-0k719                                                    |
| Storage      |                                                          500 |
| Endpoint     | https://pksqlc-0x1v2.asia-southeast2.gcp.confluent.cloud:443 |
| Status       | UP                                                           |
+--------------+--------------------------------------------------------------+


In [6]:
%%bash
export PATH=$PATH:ccloud
ccloud ksql app describe lksqlc-x08ox
ccloud service-account list

+--------------+--------------------------------------------------------------+
| Id           | lksqlc-x08ox                                                 |
| Name         | ksqlDB_app_2                                                 |
| Topic Prefix | pksqlc-8193r                                                 |
| Kafka        | lkc-0k719                                                    |
| Storage      |                                                          500 |
| Endpoint     | https://pksqlc-8193r.asia-southeast2.gcp.confluent.cloud:443 |
| Status       | UP                                                           |
+--------------+--------------------------------------------------------------+
    Id   |     Name      |          Description            
+--------+---------------+--------------------------------+
  189051 | dev-producers | dev producer apps               
  199670 | test1         | ksqlDB service account          
  199671 | test2         | ksqlDB servic

In [2]:
%%bash
export PATH=$PATH:ccloud

ccloud kafka acl create --allow --service-account 199670 --operation READ --topic 'mqtopic'
ccloud kafka acl create --allow --service-account 199670 --operation CREATE --topic 'admin_mqconnect_keyed'
ccloud kafka acl create --allow --service-account 199670 --operation WRITE --topic 'admin_mqconnect_keyed'
ccloud kafka acl create --allow --service-account 199670 --operation READ --topic 'admin_mqconnect_keyed'

  ServiceAccountId | Permission | Operation | Resource |         Name          |  Type    
+------------------+------------+-----------+----------+-----------------------+---------+
  User:199670      | ALLOW      | CREATE    | TOPIC    | admin_mqconnect_keyed | LITERAL  
  ServiceAccountId | Permission | Operation | Resource |         Name          |  Type    
+------------------+------------+-----------+----------+-----------------------+---------+
  User:199670      | ALLOW      | WRITE     | TOPIC    | admin_mqconnect_keyed | LITERAL  
  ServiceAccountId | Permission | Operation | Resource |         Name          |  Type    
+------------------+------------+-----------+----------+-----------------------+---------+
  User:199670      | ALLOW      | READ      | TOPIC    | admin_mqconnect_keyed | LITERAL  
  ServiceAccountId | Permission | Operation | Resource |  Name   |  Type    
+------------------+------------+-----------+----------+---------+---------+
  User:199670      | ALLOW 

In [ ]:
# create stream admin_connect (text varchar) with (kafka_topic='mqtopic', value_format='AVRO');

# create stream admin_mqconnect_keyed
#  with(kafka_topic='admin_mqconnect_keyed') as
# select
# EXTRACTJSONFIELD(TEXT, '$.custid') as CUSTID,
# EXTRACTJSONFIELD(TEXT, '$.custname') as CUSTNAME,
# EXTRACTJSONFIELD(TEXT, '$.account.id') as ACCOUNTID,
# EXTRACTJSONFIELD(TEXT, '$.account.type') as ACCOUNTTYPE
# from admin_connect
# partition by EXTRACTJSONFIELD(TEXT, '$.orderId')
# emit changes;

# select * from mq_text_json_payload_keyed emit changes;